In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


I choose a dataset on data.gouv.fr : https://www.data.gouv.fr/datasets/pathologies-effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/community-resources

This dataset provides information on the number of patients treated for various pathologies, chronic treatments, or episodes of care covered by the French national health insurance system. It includes data from 2015 to 2023, updated in July 2025. 

The dataset is broken down by pathology, treatment type, age group, sex, region, and department. It also includes reimbursed expenses for people without the listed pathologies or treatments, as well as the prevalence of each condition (the proportion of patients affected in the population). 

The population used corresponds to all beneficiaries of compulsory health insurance who received at least one reimbursed healthcare service or had at least one hospital stay during the year. Small patient counts (less than 11) are masked for statistical confidentiality.

In [2]:
df = pd.read_csv('data/effectifs.csv', sep=';')

In [3]:
df.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,Ntop,Npop,prev,Niveau prioritaire,libelle_classe_age,libelle_sexe,tri
0,2023,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,1,999,110.0,7290,1.523,"2,3",de 75 à 79 ans,hommes,31.0
1,2023,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,2,972,120.0,7080,1.640,"2,3",de 75 à 79 ans,hommes,31.0
2,2023,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,3,999,30.0,1590,1.638,"2,3",de 75 à 79 ans,hommes,31.0
3,2023,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,4,999,250.0,10010,2.497,"2,3",de 75 à 79 ans,hommes,31.0
4,2023,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,6,976,NaN,560,NaN,"2,3",de 75 à 79 ans,hommes,31.0


I will explain every feature to get a better understanding of them :
- annee: This is the reference year of the data.

- patho_niv1: This is the main category of the pathology or treatment. Example: "Cardioneurovascular diseases".

- patho_niv2: This is a more specific subcategory of the pathology. Example: "Valvular disease".

- patho_niv3: This is the most detailed level of the pathology or treatment (often identical to patho_niv2 if no further subdivision exists). Example: "Valvular disease".

- top: This is an internal unique code used to identify the pathology or treatment in the mapping. Example: "MCV_MVA_IND".

- cla_age_5: This is the coded age group, usually grouped in 5-year intervals. Example: "75-79".

- sexe: This is the sex code: 1 = male, 2 = female (generally).

- region: This is the code of the administrative region.

- dept: This is the code of the administrative department.

- Ntop: This is the number of patients treated for this pathology in the area. Values <11 are masked as "NS" (Not Significant) to protect confidentiality.

- Npop: This is the total reference population for the area and age group.

- prev: This is the prevalence, i.e., the percentage of patients affected by the pathology within the reference population.

- Niveau prioritaire: This indicates the priority level or internal hierarchical category, often used for data visualization (e.g., "2,3").

- libelle_classe_age: This is the readable label for the age class. Example: "from 75 to 79 years old".

- libelle_sexe: This is the readable label for sex. Example: "men" or "women".

- tri: This is a sorting variable used for display or ranking in visualizations.

In [4]:
df.shape

(5216400, 16)

This dataset is way too big, we will have to remove some fetures and lines.

In [5]:
df.columns

Index(['annee', 'patho_niv1', 'patho_niv2', 'patho_niv3', 'top', 'cla_age_5',
       'sexe', 'region', 'dept', 'Ntop', 'Npop', 'prev', 'Niveau prioritaire',
       'libelle_classe_age', 'libelle_sexe', 'tri'],
      dtype='object')

Let's see and remove null lines because we don't need to make this complicated as we have too much data.

In [6]:
df.isnull().sum()

annee                       0
patho_niv1                  0
patho_niv2             544320
patho_niv3            1179360
top                         0
cla_age_5                   0
sexe                        0
region                      0
dept                        0
Ntop                  1382435
Npop                        0
prev                  1382435
Niveau prioritaire      68040
libelle_classe_age          0
libelle_sexe                0
tri                     68040
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.shape

(2784685, 16)

This is a better size but we can do more. First let's change this year into a real date, this will be helpful after.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2784685 entries, 0 to 5216399
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   annee               int64  
 1   patho_niv1          object 
 2   patho_niv2          object 
 3   patho_niv3          object 
 4   top                 object 
 5   cla_age_5           object 
 6   sexe                int64  
 7   region              int64  
 8   dept                object 
 9   Ntop                float64
 10  Npop                int64  
 11  prev                float64
 12  Niveau prioritaire  object 
 13  libelle_classe_age  object 
 14  libelle_sexe        object 
 15  tri                 float64
dtypes: float64(3), int64(4), object(9)
memory usage: 361.2+ MB


In [10]:
# change df["annee"] to datetime
df["annee"] = pd.to_datetime(df["annee"], format='%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2784685 entries, 0 to 5216399
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   annee               datetime64[ns]
 1   patho_niv1          object        
 2   patho_niv2          object        
 3   patho_niv3          object        
 4   top                 object        
 5   cla_age_5           object        
 6   sexe                int64         
 7   region              int64         
 8   dept                object        
 9   Ntop                float64       
 10  Npop                int64         
 11  prev                float64       
 12  Niveau prioritaire  object        
 13  libelle_classe_age  object        
 14  libelle_sexe        object        
 15  tri                 float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(9)
memory usage: 361.2+ MB


Let's reduc the number of lines by starting in 2021 where the Covid pandemic was really huge.

In [11]:
# now i want to start from 2021
df = df[df["annee"] >= "2015-01-01"]

In [12]:
df["annee"].value_counts()

annee
2022-01-01    316017
2023-01-01    315239
2021-01-01    314968
2020-01-01    312635
2019-01-01    307285
2018-01-01    306341
2017-01-01    305376
2016-01-01    304117
2015-01-01    302707
Name: count, dtype: int64

In [13]:
df["sexe"].value_counts()

sexe
9    1012114
2     891816
1     880755
Name: count, dtype: int64

In [14]:
df.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,Ntop,Npop,prev,Niveau prioritaire,libelle_classe_age,libelle_sexe,tri
0,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,1,999,110.0,7290,1.523,"2,3",de 75 à 79 ans,hommes,31.0
1,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,2,972,120.0,7080,1.640,"2,3",de 75 à 79 ans,hommes,31.0
2,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,3,999,30.0,1590,1.638,"2,3",de 75 à 79 ans,hommes,31.0
3,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,4,999,250.0,10010,2.497,"2,3",de 75 à 79 ans,hommes,31.0
6,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,MCV_MVA_IND,75-79,1,11,75,940.0,35700,2.642,"2,3",de 75 à 79 ans,hommes,31.0


There is 9 values in the sexe column which references basically to unknow sex so we delete it.
Also the department 99 is there for not specified departments so we delete those rows

In [16]:
df = df[df["sexe"] != 9]
df = df[df["dept"] != 99]

The top column is only a code for the 3 levels of patho, we don't need it.

Niveau prioritaire is a intern priority level, we don't need it.

We already have sex with 1 and zeros so we don't need libelle_sexe.

Same for libelle_classe_age, we already have cla_age_5.

Finally tri is a sorting variable for visualisation in intern but we are going to create our own visualisations so we don't need it

In [17]:
df.drop(columns = 'top', inplace=True)
df.drop(columns = 'Niveau prioritaire', inplace=True)
df.drop(columns = 'libelle_classe_age', inplace=True)
df.drop(columns = 'libelle_sexe', inplace=True)
df.drop(columns = 'tri', inplace=True)

df.head()

,annee,patho_niv1,patho_niv2,patho_niv3,cla_age_5,sexe,region,dept,Ntop,Npop,prev
0,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,75-79,1,1,999,110.0,7290,1.523
1,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,75-79,1,2,972,120.0,7080,1.640
2,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,75-79,1,3,999,30.0,1590,1.638
3,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,75-79,1,4,999,250.0,10010,2.497
6,2023-01-01,Maladies cardioneurovasculaires,Maladie valvulaire,Maladie valvulaire,75-79,1,11,75,940.0,35700,2.642


Let's export our new clean csv.

In [18]:
df.to_csv('data/effectifs_cleaned.csv', index=False)